# The Goal
Start by examining the data dictionary for the SQL database we'll be working with, which comes from this kaggle page. Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database.

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

The name of the team
The total number of goals scored by the team during the 2011 season
The total number of wins the team earned during the 2011 season
A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
The team's win percentage on days where it was raining during games in the 2011 season.

# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pdb

import sqlite3 
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

import requests

from datetime import date, datetime
import time
import pymongo
import os

import dotenv
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
sns.set(style='darkgrid')
sns.set_context('poster')

In [3]:
!pip install python-dotenv

 # Inspecting Data

In [5]:
cur.execute('''SELECT * FROM matches;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(20)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [10]:
# Inspecting the Teams' table
cur.execute('''SELECT * FROM teams;''')
teams_df = pd.DataFrame(cur.fetchall())
teams_df.columns = [x[0] for x in cur.description]
teams_df.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [11]:
teams_df.shape

(468, 8)

In [14]:
teams_df['TeamName'].unique()

array(['Bayern Munich', 'Dortmund', 'Leverkusen', 'RB Leipzig',
       'Schalke 04', "M'gladbach", 'Wolfsburg', 'FC Koln', 'Hoffenheim',
       'Hertha', 'Mainz', 'Hamburg', 'Werder Bremen', 'Ein Frankfurt',
       'Augsburg', 'Freiburg', 'Stuttgart', 'Hannover', 'Ingolstadt',
       'Darmstadt', 'Paderborn', 'Nurnberg', 'Braunschweig',
       'Greuther Furth', 'Fortuna Dusseldorf', 'Kaiserslautern',
       'St Pauli', 'Bochum', 'Karlsruhe', 'Bielefeld', 'Cottbus',
       'Duisburg', 'Hansa Rostock', 'Aachen', 'Union Berlin',
       'Heidenheim', 'Dresden', 'Sandhausen', 'Erzgebirge Aue', 'Kiel',
       'Regensburg', 'Munich 1860', 'Wurzburger Kickers', 'Frankfurt FSV',
       'Aalen', 'Oberhausen', 'Osnabruck', 'Koblenz', 'Ahlen', 'Wehen',
       'CZ Jena', 'Offenbach', 'RW Essen', 'Burghausen', 'Unterhaching',
       'Saarbrucken', 'Siegen'], dtype=object)

In [15]:
teams_df['TeamName'].count

<bound method Series.count of 0      Bayern Munich
1           Dortmund
2         Leverkusen
3         RB Leipzig
4         Schalke 04
           ...      
463        Paderborn
464     Unterhaching
465        Offenbach
466           Siegen
467     Braunschweig
Name: TeamName, Length: 468, dtype: object>

In [17]:
# Accessing the unique team names on the table 
cur.execute('''SELECT * FROM unique_teams;''')
unique_teams_df = pd.DataFrame(cur.fetchall())
unique_teams_df.columns = [x[0] for x in cur.description]
unique_teams_df.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5
